In [ ]:
!nvidia-smi

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

In [ ]:
!pip uninstall -y vllm
!pip install llamafactory[metrics]==0.7.1
!pip install accelerate==0.30.1

In [ ]:
# 查看llamafactory是否安装成功
!llamafactory-cli version

In [ ]:
# 检查torch和显卡
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.__version__

In [ ]:
# 下载模型
!git clone https://www.modelscope.cn/baichuan-inc/Baichuan2-7B-Chat.git


In [ ]:
%pip install modelscope

In [ ]:
!ls -lh /home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat/pytorch_model.bin

In [ ]:
# 测试模型是否导入成功
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# 本地模型路径
model_dir = "/home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat"

# 加载分词器
try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
    )
    print("Tokenizer loaded successfully!")
except Exception as e:
    print("Error loading tokenizer:", e)

# 加载模型
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
    )
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

# 加载生成配置
try:
    model.generation_config = GenerationConfig.from_pretrained(model_dir)
    print("Generation config loaded successfully!")
except Exception as e:
    print("Error loading generation config:", e)

In [ ]:
!CUDA_VISIBLE_DEVICES=0 llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path /home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat \
    --dataset train \
    --dataset_dir /home/syq/fine_tuning/test1/LLaMA-Factory/data \
    --template baichuan2 \
    --finetuning_type lora \
    --output_dir ./saves/baichuan/lora/sft \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 100 \
    --warmup_steps 500 \
    --save_steps 1000 \
    --eval_steps 1000 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --val_size 0.1 \
    --plot_loss \
    --fp16